# NLP feature extraction

In [ ]:
from pathlib import Path
from typing import List

import gensim
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pyLDAvis.gensim
import seaborn as sns
import spacy
import string

from gensim import corpora
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
from scipy.io.arff import loadarff

from src.data_preprocessing.data_preprocessor import (
    load_data,
    change_to_numeric,
    feature_engineering,
    transform_label,
    convert_float_to_int,
    drop_cols,
    filter_na,
)

In [ ]:
FULL_DATA_PATH = Path("../../data/dataset.csv")
DATA_PATH = Path("../../data/file65ef3a759daf.arff")

## Functions

In [ ]:
def preprocessor(data_path: Path, cols: List[str]) -> pd.DataFrame:
    """Load data and perform preprocessing steps.

    Parameters
    -------
    path_ : Path
            Path of the data.
    cols : List
            List of columns to drop.

    Returns
    -------
    data : pd.DataFrame
            Processed data.
    """
    # load the data
    data, _ = load_data(data_path)

    # convert yes/no to 0/1 and year to int
    data = change_to_numeric(data)
    data = convert_float_to_int(data)

    # perform feature engineering on state columns
    data = feature_engineering(data)

    # drop unwished cols
    data = drop_cols(data, cols)

    # transform label to 0/1 for citation
    data = transform_label(data)

    # filter na
    data = filter_na(data)

    return data

In [ ]:
def preprocess_text(df: pd.DataFrame, column_name: str = "Description") -> pd.DataFrame:
    """reformats text so that LDA can be applied in the next step

    Parameters
    -------
    df : pd.DataFrame
            Data to transform.
    column_name : (str, optional)
            Name of column to be transformed.
            Defaults to "Description".

    Returns
    -------
    df : pd.DataFrame
            Transformed data.
    """
    # making text lower case
    lowercase_text = df[column_name].apply(lambda x: x.lower())

    # tokenize text
    tokenized_text = lowercase_text.apply(lambda x: nltk.word_tokenize(x))

    # removing stopwords
    stop_words = set(stopwords.words("english"))
    clean_text = tokenized_text.apply(
        lambda tokens: [
            word for word in tokens if word not in stop_words and word not in string.punctuation
        ]
    )

    # lemmatize words
    lemmatized_text = clean_text.apply(
        lambda tokens: [token.lemma_ for token in nlp(" ".join(tokens))]
    )

    df["description_clean"] = lemmatized_text.apply(lambda lem_tokens: " ".join(lem_tokens))

    return df

In [ ]:
def create_n_topics(
    df: pd.DataFrame,
    column_name: str = "description_clean",
    n_topics: int = 10,
    random_state: int = 42,
) -> pd.DataFrame:
    """Applies LDA to a text column of DF and adds LDA topic distributions as new features.

    Parameters
    ----------
    df : pd.DataFrame
        Data to transform.
    column_name : str, optional
        Name of the column to be transformed. Defaults to "description_clean".
    num_topics : int, optional
        Number of topics for LDA. Defaults to 10.
    random_stat : int
            Random state of split for reproducibility.

    Returns
    -------
    df : pd.DataFrame
        Transformed data with added LDA topic features.
    """

    # Preprocess text
    df = preprocess_text(df, column_name)

    # Tokenized text is already available from preprocessing but needs to be list of list
    tokenized_text = df[column_name].apply(lambda x: x.split())

    # Create a Gensim dictionary and corpus
    dictionary = corpora.Dictionary(tokenized_text)
    corpus = [dictionary.doc2bow(text) for text in tokenized_text]

    # Train an LDA model
    lda_model = gensim.models.LdaModel(
        corpus=corpus, id2word=dictionary, num_topics=n_topics, passes=10, random_state=random_state
    )

    # Extract LDA topics
    topics = lda_model[corpus]

    for topic in lda_model[corpus]:
        print(topic)

    print(f"Number of topics extracted: {len(topics)}")

    # Add LDA topic distributions as new features
    for i in range(n_topics):
        df[f"Topic_{i+1}"] = [topic[i][1] if i < len(topic) else 0 for topic in topics]

    return df, lda_model, corpus, dictionary

In [ ]:
def threshold_transform(data_frame: pd.DataFrame, t: float, columns: list[str]) -> pd.DataFrame:
    """
    Transform specified columns in a DataFrame based on a threshold 't'.

    Parameters
    -------
        data_frame : pd.DataFrame
            The DataFrame to be transformed.
        t : float
            The threshold value.
        columns_to_transform : list
            List of column names to be transformed.

    Returns
    -------
        pandas.DataFrame: A new DataFrame with specified columns transformed.
    """
    transformed_df = data_frame.copy()

    for column in columns:
        transformed_df[column] = (transformed_df[column] >= t).astype(int)

    return transformed_df

## Preprocessed dataset

In [ ]:
data, meta = loadarff(DATA_PATH)
data = pd.DataFrame(data)

In [ ]:
print("Dimensions of the dataset:", data.shape)

In [ ]:
# remove b string from data
str_df = data.select_dtypes([object])
str_df = str_df.stack().str.decode("utf-8").unstack()
data = pd.concat([str_df, data.select_dtypes(exclude=[object])], axis=1)

In [ ]:
# apply preprocessing to data
data = preprocessor(
    DATA_PATH, ["Model", "Charge", "Driver.City", "Arrest.Type", "Commercial.Vehicle"]
)

In [ ]:
# load stopwords and en_core_web_sm used in preprocess_text
nltk.download("stopwords")
nlp = spacy.load("en_core_web_sm")

In [ ]:
data = preprocess_text(data)

In [ ]:
tokenized_text = data["description_clean"].apply(lambda x: x.split())
dictionary = corpora.Dictionary(tokenized_text)
corpus = [dictionary.doc2bow(text) for text in tokenized_text]

topic_range = range(2, 21)

models = []
coherence_scores = []

for n_topics in topic_range:
    lda_model = gensim.models.LdaModel(
        corpus=corpus, id2word=dictionary, num_topics=n_topics, passes=10
    )
    models.append(lda_model)
    coherence_model = CoherenceModel(
        model=lda_model,
        texts=data["description_clean"].str.split(),
        dictionary=dictionary,
        coherence="c_v",
    )
    coherence_scores.append(coherence_model.get_coherence())

In [ ]:
# Plot the coherence scores
plt.plot(topic_range, coherence_scores)
plt.xticks(np.arange(2, 21, step=1))
plt.xlabel("Number of Topics")
plt.ylabel("Coherence Score")
plt.title("Coherence Score vs. Number of Topics")
plt.show()

In [ ]:
data, lda_model, corpus, dictionary = create_n_topics(
    data, column_name="description_clean", n_topics=3
)

In [ ]:
print(data.head())

In [ ]:
for i in range(3):
    print(i)

In [ ]:
plt.figure(figsize=(8, 6))
for i in range(3):
    plt.hist(data[f"Topic_{i+1}"], alpha=0.5, label=f"Topic_{i+1}", bins=20)

plt.xlabel("Values")
plt.ylabel("Frequency")
plt.title("Distribution of Topics")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# visualing the topic seperation using pyLDAvis
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

In [ ]:
data = threshold_transform(data, t=0.334, columns=["Topic_1", "Topic_2", "Topic_3"])

In [ ]:
data.head()

In [ ]:
df_numeric = data.select_dtypes(exclude=[object])
# shows correlation of numeric features
corr = df_numeric.corr()
corr

# create heat map of correlation plot
sns.heatmap(corr)
plt.title("Correlation plot of numerical features")